In [63]:
%%bash
cd /home/jovyan
#cd /srv/conda/envs/notebook/lib/python3.7/site-packages/beakerx/kernel/clojure/

java -cp "/srv/conda/envs/notebook/lib/python3.7/site-packages/beakerx/kernel/clojure/lib/core.specs.alpha-0.1.24.jar:/srv/conda/envs/notebook/lib/python3.7/site-packages/beakerx/kernel/clojure/lib/clojure-1.9.0.jar:/srv/conda/envs/notebook/lib/python3.7/site-packages/beakerx/kernel/clojure/lib/spec.alpha-0.1.143.jar" clojure.main -e ":ok"

 

:ok



In [ ]:
%classpath add mvn scicloj scicloj.ml "0.1.0-beta3"

In [ ]:
(require '[scicloj.ml.core :as ml]
         '[scicloj.ml.metamorph :as mm]
         '[scicloj.ml.dataset :as ds])

In [ ]:
(def titanic-train
  (->
   (ds/dataset "https://github.com/scicloj/metamorph-examples/raw/main/data/titanic/train.csv"
               {:key-fn keyword
                :parser-fn :string})))
                
(def titanic-test
  (->
   (ds/dataset "https://github.com/scicloj/metamorph-examples/raw/main/data/titanic/test.csv"
               {:key-fn keyword
                :parser-fn :string})
   (ds/add-column :Survived [""] :cycle)))
   
   
;; construct pipeline function including Logistic Regression model
(def pipe-fn
  (ml/pipeline
   (mm/select-columns [:Survived :Pclass ])
   (mm/add-column :Survived (fn [ds] (map #(case % "1" "yes" "0" "no" nil "") (:Survived ds))))
   (mm/categorical->number [:Survived :Pclass])
   (mm/set-inference-target :Survived)
   (mm/model {:model-type :smile.classification/logistic-regression})))
   
;;  execute pipeline with train data including model in mode :fit
(def trained-ctx
  (pipe-fn {:metamorph/data titanic-train
            :metamorph/mode :fit}))


;; execute pipeline in mode :transform with test data which will do a prediction 
(def test-ctx
  (pipe-fn
   (assoc trained-ctx
          :metamorph/data titanic-test
          :metamorph/mode :transform)))

;; extract prediction from pipeline function result
(-> test-ctx :metamorph/data
    (ds/column-values->categorical :Survived))